#Модули


In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.datasets import load_iris

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.preprocessing import LabelEncoder

from scipy.io import arff


from sklearn.metrics import classification_report

# A) 

## Генерирање на тренинг/тест множество
Нема точки кои што се над/под кривата, сите точки лежат на кривата. 
Поради тоа, дадов threshold 0.5, така што  
**Ако y>0.5 тогаш класифицирај како 1**  
**Ако y<=0.5 тогаш класифицирај како 0**

2 линија - листа од x координати со вредности меѓу 0 и 1  
3 линија - листа каде што ги енкодираме вредностите на y во 0 или 1 во зависност од исполнетоста на условот

In [ ]:
def generate_dataset_A(train_size, test_size):
  x_train = np.random.ranf(train_size)
  y_train = np.where((np.sin(6*x_train)/6) + 0.6>0.5, 1, 0)

  x_test = np.random.ranf(test_size)
  y_test = np.where((np.sin(6*x_test)/6) + 0.6>0.5, 1, 0)
  return x_train, x_test, y_train, y_test

In [ ]:
x_train_100, x_test_50, y_train_100, y_test_50 = generate_dataset_A(100,50)
x_train_1000, x_test_300, y_train_1000, y_test_300 = generate_dataset_A(1000,300)
x_train_10000, x_test_3000, y_train_10000, y_test_3000 = generate_dataset_A(10000,3000)

## Модел со KNN класификација

Со ова парче код испробуваме 3 вредности за neighbors при креирање на моделот KNN класификација и за истото извршуваме тестирање со тоа што се печатат мерките accuracy, recall, precision и f1 score.

Согледуваме дека сегде е 1.0 при предвидувањето, бидејќи е hard coded искодирано, односно самите го поставивме threshold-от, со тоа му олеснуваме многу на моделот да предвиди точно.

In [ ]:
for k_neighbors in [1,3,7]:
  model = KNeighborsClassifier(n_neighbors=k_neighbors)

  model.fit(x_train_100.reshape(-1,1),y_train_100.ravel())
  y_pred_50 = model.predict(x_test_50.reshape(-1,1))
  
  model.fit(x_train_1000.reshape(-1,1),y_train_1000.ravel())
  y_pred_300 = model.predict(x_test_300.reshape(-1,1))

  model.fit(x_train_10000.reshape(-1,1),y_train_10000.ravel())
  y_pred_3000 = model.predict(x_test_3000.reshape(-1,1))

  print("K =",k_neighbors)
  print(classification_report(y_test_50,y_pred_50))
  print(classification_report(y_test_300,y_pred_300))
  print(classification_report(y_test_3000,y_pred_3000))
  print("\n\n")

K = 1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        39

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        74
           1       1.00      1.00      1.00       226

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       941
           1       1.00      1.00      1.00      2059

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000




K = 3
    

## Модел со Decision Tree 

Ги менуваме вредностите на криетериумот при поделба, односно Џини индекс или ентропија и на максималната длабочина на дрвото при поделба, всушност пробуваме со длабочина од 10 и од 100.  
Pattern-от лесно е најден од дрвото на одлука, затоа и имаме максимален број на предвидувања во сите примероци.

In [ ]:
for criteria in ['gini','entropy']:
  for depth in [10,100]:
    model = DecisionTreeClassifier(criterion=criteria, max_depth=depth,random_state=42)
    
    model.fit(x_train_100.reshape(-1,1),y_train_100.ravel())
    y_pred_50 = model.predict(x_test_50.reshape(-1,1))

    model.fit(x_train_1000.reshape(-1,1),y_train_1000.ravel())
    y_pred_300 = model.predict(x_test_300.reshape(-1,1))

    model.fit(x_train_10000.reshape(-1,1),y_train_10000.ravel())
    y_pred_3000 = model.predict(x_test_3000.reshape(-1,1))

    print("CRITERION=",criteria,"\tMAX DEPTH=",depth)
    print(classification_report(y_test_50,y_pred_50))
    print(classification_report(y_test_300,y_pred_300))
    print(classification_report(y_test_3000,y_pred_3000))
    print("\n\n")

CRITERION= gini 	MAX DEPTH= 10
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        39

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        74
           1       1.00      1.00      1.00       226

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       941
           1       1.00      1.00      1.00      2059

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00

# Б)

## Генерирање на тренинг/тест множество

На почетокот [од 2 до 4 линија] генерираме random случајни и y координати со опсег од 0 до 1.  
После за секоја точка гледаме дали припаѓа во опсезите за црн или бел квадрат.  
Тоа го правиме со помош на координатниот систем. Пример:  

Ако имаме генерирано точка (0.42, 0.77), и сега ги проверуваме вредностите за границите кои се дефинирани за дали  
x = 0.42 се наоѓа меѓу 0.25 и 0.5, уствари тоа е втората колона  
y = 0.77 се наоѓа меѓу 0.75 и 1.0, всушност тоа е првиот ред од шаховската табла.  

Значи гледаме на полето во шаховската табла b4 и тоа е црно поле.

In [ ]:
def generate_dataset_B(train_size, test_size):
  x_cord = np.random.ranf(train_size)
  y_cord = np.random.ranf(train_size)
  x_train = [list(point) for point in zip(x_cord,y_cord)]
  y_train = list()

  for point in x_train:
    if (0<point[0]<0.25 and 0<point[1]<0.25) or (0.5<point[0]<0.75 and 0<point[1]<0.25) or (0.25<point[0]<0.50 and 0.25<point[1]<0.50) or (0.75<point[0]<1 and 0.25<point[1]<0.50) or (0<point[0]<0.25 and 0.50<point[1]<0.75) or (0.50<point[0]<0.75 and 0.50<point[1]<0.75) or (0<point[0]<0.25 and 0.50<point[1]<0.75) or (0.25<point[0]<0.5 and 0.75<point[1]<1.0) or (0.75<point[0]<1.0 and 0.75<point[1]<1.0):
      y_train.append(1)
    else:
      y_train.append(0)

  x_cord = np.random.ranf(test_size)
  y_cord = np.random.ranf(test_size)

  x_test = [list(point) for point in zip(x_cord,y_cord)]
  y_test = list()

  for point in x_test:
    if (0<point[0]<0.25 and 0<point[1]<0.25) or (0.5<point[0]<0.75 and 0<point[1]<0.25) or (0.25<point[0]<0.50 and 0.25<point[1]<0.50) or (0.75<point[0]<1 and 0.25<point[1]<0.50) or (0<point[0]<0.25 and 0.50<point[1]<0.75) or (0.50<point[0]<0.75 and 0.50<point[1]<0.75) or (0<point[0]<0.25 and 0.50<point[1]<0.75) or (0.25<point[0]<0.5 and 0.75<point[1]<1.0) or (0.75<point[0]<1.0 and 0.75<point[1]<1.0):
      y_test.append(1)
    else:
      y_test.append(0)

  return x_train, x_test, y_train, y_test


In [ ]:
x_train_100, x_test_50, y_train_100, y_test_50 = generate_dataset_B(100,50)
x_train_1000, x_test_300, y_train_1000, y_test_300 = generate_dataset_B(1000,300)
x_train_10000, x_test_3000, y_train_10000, y_test_3000 = generate_dataset_B(10000,3000)

## Модел со KNN класификација

Овде можеме да видиме без разлика на параметарот за n соседи, не го менува резултатот во третото множество каде што тренирачкото се состои од 10000 примероци. Што значи овде гледаме дека колку повеќе примероци има за тренинг толку подобри резултати имаме во предвидувањето.

In [ ]:
for k_neighbors in [1,3,7]:
  model = KNeighborsClassifier(n_neighbors=k_neighbors)

  model.fit(x_train_100,y_train_100)
  y_pred_50 = model.predict(x_test_50)
  
  model.fit(x_train_1000,y_train_1000)
  y_pred_300 = model.predict(x_test_300)

  model.fit(x_train_10000,y_train_10000)
  y_pred_3000 = model.predict(x_test_3000)

  print("K =",k_neighbors)
  print(classification_report(y_test_50,y_pred_50))
  print(classification_report(y_test_300,y_pred_300))
  print(classification_report(y_test_3000,y_pred_3000))
  print("\n\n")

K = 1
              precision    recall  f1-score   support

           0       0.79      0.88      0.84        26
           1       0.86      0.75      0.80        24

    accuracy                           0.82        50
   macro avg       0.83      0.82      0.82        50
weighted avg       0.82      0.82      0.82        50

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       169
           1       0.92      0.92      0.92       131

    accuracy                           0.93       300
   macro avg       0.93      0.93      0.93       300
weighted avg       0.93      0.93      0.93       300

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1493
           1       0.98      0.98      0.98      1507

    accuracy                           0.98      3000
   macro avg       0.98      0.98      0.98      3000
weighted avg       0.98      0.98      0.98      3000




K = 3
    

## Модел со Decision Tree
Истото го гледаме и овде.
Со тоа што плус овде резултатите остануваат исти дури и кога тренирачкото множество е >1000 примероци. Не прави голема разлика многу критериумот за поделба во комбинација со максималната длабочина на дрвото во однос на резултатите.

In [ ]:
for criteria in ['gini','entropy']:
  for depth in [10,100]:
    model = DecisionTreeClassifier(criterion=criteria, max_depth=depth,random_state=42)
    
    model.fit(x_train_100,y_train_100)
    y_pred_50 = model.predict(x_test_50)
    
    model.fit(x_train_1000,y_train_1000)
    y_pred_300 = model.predict(x_test_300)

    model.fit(x_train_10000,y_train_10000)
    y_pred_3000 = model.predict(x_test_3000)

    print("CRITERION=",criteria,"\tMAX DEPTH=",depth)
    print(classification_report(y_test_50,y_pred_50))
    print(classification_report(y_test_300,y_pred_300))
    print(classification_report(y_test_3000,y_pred_3000))
    print("\n\n")

CRITERION= gini 	MAX DEPTH= 10
              precision    recall  f1-score   support

           0       0.67      0.85      0.75        26
           1       0.76      0.54      0.63        24

    accuracy                           0.70        50
   macro avg       0.72      0.69      0.69        50
weighted avg       0.71      0.70      0.69        50

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       169
           1       0.97      0.98      0.98       131

    accuracy                           0.98       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300

              precision    recall  f1-score   support

           0       0.95      0.88      0.91      1493
           1       0.89      0.95      0.92      1507

    accuracy                           0.91      3000
   macro avg       0.92      0.91      0.91      3000
weighted avg       0.92      0.91      0.91

# В) 


## Лоадирање на податоците

In [ ]:
dataset = load_iris()
X = dataset.data
y = dataset.target

## Модел со KNN класификација
[Во една листа се претставени точностите при предвидувањето за секој Fold]

За секој Fold гледаме каква е точноста на предвидувањето со KNN, затоа гледаме 10 елементи во листата што е испечатена.

За секој параметар гледаме дека некои точности како да си менуваат местата во некои од Fold-овите. Но остануваат истите 0.93, 0.86 и 1.   
Од прв поглед, можам да претпоставам дека некои примероци се „проблематични“ што само овие бројки ги перципираме.

In [ ]:
for k_neighbors in [1,3,7]:
  model = KNeighborsClassifier(n_neighbors=k_neighbors)
  print("K =",k_neighbors)
  print(cross_val_score(model,X,y,cv=10,scoring='accuracy'))
  print("\n\n")

K = 1
[1.         0.93333333 1.         0.93333333 0.86666667 1.
 0.86666667 1.         1.         1.        ]



K = 3
[1.         0.93333333 1.         0.93333333 0.86666667 1.
 0.93333333 1.         1.         1.        ]



K = 7
[1.         0.93333333 1.         1.         0.86666667 0.93333333
 0.93333333 1.         1.         1.        ]





## Модел со Decision Tree

In [ ]:
for criteria in ['gini','entropy']:
  for depth in [10,100]:
    model = DecisionTreeClassifier(criterion=criteria, max_depth=depth,random_state=42)

    print("CRITERION=",criteria,"\tMAX DEPTH=",depth)
    print(cross_val_score(model,X,y,cv=10,scoring='accuracy'))
    print("\n\n")

CRITERION= gini 	MAX DEPTH= 10
[1.         0.93333333 1.         0.93333333 0.93333333 0.86666667
 0.93333333 0.93333333 1.         1.        ]



CRITERION= gini 	MAX DEPTH= 100
[1.         0.93333333 1.         0.93333333 0.93333333 0.86666667
 0.93333333 0.93333333 1.         1.        ]



CRITERION= entropy 	MAX DEPTH= 10
[1.         0.93333333 1.         0.93333333 0.93333333 0.86666667
 0.93333333 0.93333333 1.         1.        ]



CRITERION= entropy 	MAX DEPTH= 100
[1.         0.93333333 1.         0.93333333 0.93333333 0.86666667
 0.93333333 0.93333333 1.         1.        ]





# Г)

## Енкодирање на податоците

При вчитувањето на податоците, таргет класата е текстуална, па потребно е енкодирање нејзино.

In [ ]:
data = arff.loadarff('seismic-bumps.arff')
df = pd.DataFrame(data[0])
df.head()

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,ghazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
0,b'a',b'a',b'N',15180.0,48.0,-72.0,-72.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
1,b'a',b'a',b'N',14720.0,33.0,-70.0,-79.0,b'a',1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2000.0,2000.0,b'0'
2,b'a',b'a',b'N',8050.0,30.0,-81.0,-78.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
3,b'a',b'a',b'N',28820.0,171.0,-23.0,40.0,b'a',1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3000.0,3000.0,b'0'
4,b'a',b'a',b'N',12640.0,57.0,-63.0,-52.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'


In [ ]:
df = df.apply(LabelEncoder().fit_transform)
X = df.drop(columns=['class'],axis=1)
y = df['class']

## Модел со KNN класификација

Овде најдобриот модел е во случаевите кога K=3,7 во 5-тиот Fold со точност >0.93

In [ ]:
for k_neighbors in [1,3,7]:
  model = KNeighborsClassifier(n_neighbors=k_neighbors)
  print("K =",k_neighbors)
  print(cross_val_score(model,X,y,cv=10,scoring='accuracy'))
  print("\n\n")

K = 1
[0.86100386 0.63320463 0.69498069 0.92277992 0.91860465 0.87209302
 0.89534884 0.8875969  0.89534884 0.90310078]



K = 3
[0.86486486 0.73745174 0.77220077 0.93436293 0.93023256 0.93023256
 0.90697674 0.93023256 0.92635659 0.92635659]



K = 7
[0.84555985 0.78764479 0.81467181 0.93436293 0.93410853 0.93410853
 0.93410853 0.93410853 0.93410853 0.93410853]





## Модел со Decision Tree

Овде е интересното што најдобриот модел од Decision Tree е со параметарот max depth = 10 и без разлика дали критериумот за поделба на дрвото е gini или ентропија

In [ ]:
for criteria in ['gini','entropy']:
  for depth in [10,100]:
    model = DecisionTreeClassifier(criterion=criteria, max_depth=depth,random_state=42)

    print("CRITERION=",criteria,"\tMAX DEPTH=",depth)
    print(cross_val_score(model,X,y,cv=10,scoring='accuracy'))
    print("\n\n")

CRITERION= gini 	MAX DEPTH= 10
[0.85328185 0.67953668 0.77220077 0.91505792 0.93023256 0.91472868
 0.91860465 0.92635659 0.90697674 0.93023256]



CRITERION= gini 	MAX DEPTH= 100
[0.84942085 0.67181467 0.75289575 0.84942085 0.90310078 0.89147287
 0.87209302 0.90697674 0.88372093 0.91860465]



CRITERION= entropy 	MAX DEPTH= 10
[0.86486486 0.74903475 0.82625483 0.91891892 0.93023256 0.91085271
 0.91860465 0.93023256 0.92248062 0.91472868]



CRITERION= entropy 	MAX DEPTH= 100
[0.86486486 0.69111969 0.77606178 0.91505792 0.8875969  0.86046512
 0.91472868 0.8875969  0.89534884 0.89147287]





# Д)

## Креирање на податочното множество и поделбата негова на тренинг и тест множеството

Ги читаме податоците од датотека, ги средуваме со цел да добиеме features и таргет класата.
Ги енкодираме y вредностите, бидејќи се букви со бројки
од 0-25 соодветно со редоследот во латиницата.  
На крај го делиме множеството така што големината на тренингот е 2/3, додека пак тестирачкото е 1/3

In [ ]:
with open('/content/letter-recognition.data') as f:
    lines = f.readlines()
dataset = [ line.split() for line in lines ]

In [ ]:
y = [line[0][0] for line in dataset]
X = [list(map(int,line[0].split(",")[1:])) for line in dataset]

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=1/3, train_size=2/3, random_state=42 )

## Модел со KNN класификација

Овде се покажа дека KNN е подобро да се искористи, отколку DecisionTree, бидејќи резултатите се значително подобри
каде што точноста со KNN e 95%, додека Decision Tree e 88%

In [ ]:
for k_neighbors in [1,3,7]:
  model = KNeighborsClassifier(n_neighbors=k_neighbors)
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  print("K =",k_neighbors)
  print(classification_report(y_test,y_pred))
  print("\n\n")

K = 1
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       267
           1       0.89      0.92      0.90       256
           2       0.97      0.95      0.96       214
           3       0.94      0.96      0.95       284
           4       0.96      0.94      0.95       261
           5       0.92      0.95      0.94       235
           6       0.95      0.94      0.95       247
           7       0.89      0.83      0.86       241
           8       0.95      0.95      0.95       244
           9       0.95      0.94      0.94       248
          10       0.87      0.93      0.90       204
          11       0.98      0.98      0.98       249
          12       1.00      0.98      0.99       276
          13       0.97      0.94      0.96       262
          14       0.93      0.96      0.94       251
          15       0.96      0.93      0.95       280
          16       0.96      0.95      0.95       281
          17       0.

## Модел со Decision Tree

In [ ]:
for criteria in ['gini','entropy']:
  for depth in [10,100]:
    model = DecisionTreeClassifier(criterion=criteria, max_depth=depth,random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("CRITERION=",criteria,"\tMAX DEPTH=",depth)
    print(classification_report(y_test,y_pred))
    print("\n\n")

CRITERION= gini 	MAX DEPTH= 10
              precision    recall  f1-score   support

           0       0.97      0.82      0.89       267
           1       0.63      0.65      0.64       256
           2       0.35      0.77      0.48       214
           3       0.59      0.79      0.68       284
           4       0.75      0.55      0.64       261
           5       0.72      0.71      0.72       235
           6       0.43      0.73      0.54       247
           7       0.34      0.32      0.33       241
           8       0.87      0.81      0.84       244
           9       0.98      0.76      0.86       248
          10       0.57      0.70      0.63       204
          11       0.92      0.78      0.84       249
          12       0.92      0.83      0.87       276
          13       0.89      0.79      0.84       262
          14       0.57      0.62      0.59       251
          15       0.62      0.80      0.70       280
          16       0.54      0.65      0.59       